In [24]:
import pandas as pd
import numpy as np
import torch

In [25]:
df = pd.read_pickle("data/large_dataset/correct_trajectories.pkl")
label_dict = {genre: idx for idx, genre in enumerate(df["genre"].unique())}
df["name"] = df["file"].apply(lambda x: x.split("/")[-1])

In [26]:
trajs = np.concatenate(df["trajectory"].to_numpy()).reshape(-1, 10, 768)
names = df["name"].to_numpy()
genres = df["genre"].to_numpy()
labels = np.array([label_dict[genre] for genre in genres])

In [27]:
liked_names = [
    "data/large_dataset/80s_pop/Culture Club - Karma Chameleon (Official Music Video).mp3",
    "data/large_dataset/80s_pop/Daryl Hall & John Oates - You Make My Dreams (Official HD Video).mp3",
    "data/large_dataset/90s_hiphop/Wu-Tang Clan - C.R.E.A.M. (Official HD Video).mp3",
    "data/large_dataset/2000s_alt_rock/MGMT - Electric Feel (Official HD Video).mp3",
    "data/large_dataset/2000s_alt_rock/Red Hot Chili Peppers - Californication (Official Music Video) [HD UPGRADE].mp3",
    "data/large_dataset/2000s_alt_rock/Green Day - 21 Guns [Official Music Video].mp3",
    "data/large_dataset/2000s_rnb/Dr. Dre - Still D.R.E. ft. Snoop Dogg.mp3",
    "data/large_dataset/2010s_hiphop/Outkast - Hey Ya! (Official HD Video).mp3",
    "data/large_dataset/2010s_pop/The Weeknd - I Feel It Coming ft. Daft Punk (Official Video).mp3",
    "data/large_dataset/2010s_pop/Icona Pop - I Love It (feat. Charli XCX) [OFFICIAL VIDEO].mp3"
]

# get all the trajectories for the liked songs from df
liked = np.zeros((10, 10, 768))
for i, name in enumerate(liked_names):
    liked[i] = df[df["file"] == name]["trajectory"].iloc[0]



In [95]:

S = 10

projections = torch.zeros((S, 768, 768)).to('cuda')
for songlet in range(S):
    A = torch.tensor(liked[songlet].T)
    P = A @ torch.linalg.inv((A.T  @ A)) @ A.T
    projections[songlet] = P

trajs = torch.tensor(trajs).to('cuda').float()

projected_songs = torch.zeros_like(trajs).to('cuda')
# test_projected = np.zeros(trajs.shape)
for song in range(trajs.shape[0]):
    projected_songs[song, :, :] = torch.matmul(projections, trajs[song, :, :, None]).squeeze(-1)
    # for songlet in range(S):
    #     proj = projections[songlet] @ trajs[song, songlet, :].cpu().numpy()
    #     test_projected[song, songlet, :] = proj.cpu().numpy()

norms = torch.norm((projected_songs - trajs), dim=2, p=2)
norms = torch.sum(norms, dim=1)
_, indices = torch.sort(norms, descending=False)

/tmp/ipykernel_221318/729346365.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trajs = torch.tensor(trajs).to('cuda').float()


In [97]:
(((projected_songs.cpu().numpy() - test_projected)) < 1e-6).all()

True

In [88]:
test_projected

array([[[ 1.71662795e-02,  1.20163759e-01, -7.52040571e-03, ...,
          3.16209413e-02,  1.04000703e-02,  4.73817632e-02],
        [ 5.74216333e-03,  1.11221152e-01, -5.38646326e-03, ...,
          1.04357192e-02,  3.64602294e-03,  4.77864106e-02],
        [ 3.06438674e-03,  1.13538607e-01, -3.10798987e-03, ...,
          1.95053729e-02, -2.30626981e-03,  3.06339066e-02],
        ...,
        [-7.66425553e-03,  1.12982334e-01, -3.28348117e-03, ...,
          1.50592402e-02,  2.53033171e-02,  3.60859051e-02],
        [-4.15818992e-03,  1.08898357e-01, -2.53862109e-03, ...,
          2.59745643e-02,  7.49649499e-03,  2.47891060e-02],
        [-5.31231430e-03,  1.12038625e-01,  2.60447137e-03, ...,
          2.40933087e-02,  4.55014151e-03,  2.52892498e-02]],

       [[-7.83285096e-04,  1.13317639e-01, -5.46388898e-04, ...,
          1.29219945e-02,  1.72969052e-02,  3.95846839e-02],
        [ 1.18066044e-02,  1.14307357e-01, -5.32846524e-04, ...,
          8.39496195e-03,  1.37434444e

In [80]:
# projected = np.matmul(projs, trajs[0, :, :, None]).squeeze(-1)
# projected.shape

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
torch.matmul(torch.tensor(projs).to('cuda'), torch.tensor(trajs[:, :, :, None]).to('cuda')).squeeze(-1).shape

RuntimeError: CUDA out of memory. Tried to allocate 127.44 GiB (GPU 0; 7.92 GiB total capacity; 130.96 MiB already allocated; 7.05 GiB free; 132.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF